# Quantitative Research Case Study

In this research, we are going to consider a multiperiod binomial asset model for an asset S with N periods. Under this particular model, we have the following assumptions:

1. the initial price of the asset is $S_{0}$ = 1;
2. under the risk-neutral measure, the asset price at period j is $S_{j} = (1+v)S_{j-1}$ with probability 1/2 and $S_{j} = (1-v)S_{j-1}$ with probability 1/2, with $0 < v < 1$; and
3. the interest rate associated with borrowing/lending currency for a single time period is 0.
__________________________________________________________________________

#### Import Libraries

In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import root

#### Q1. European Call Option

In [2]:
def binomial_european(S0, K, T, N, v, p_u=1/2, r=0, call=True):
    """
    returns european call/put option value for given input 
    Parameters:
        S0: initial underlying price
        K: strike
        T: time until maturity
        N: number of time steps
        v: used to calculate up or down factor; (1+v) or (1-v) respectively 
        p_u: probability of up factor
        r: interest rate (in decimals)
        call: True for call option, False for put option
    """
    # check if 0 < v < 1:
    if v <= 0 or v >= 1:
        raise Exception('v should be >0 and <1')

    # check if 0 < p_u < 1:
    if p_u <= 0 or p_u >= 1:
        raise Exception('p_u should be >0 and <1')
    
    # compute constants used for pricing:
    dt = T/N
    u = 1+v
    d = 1-v
    p_d = 1 - p_u
    disct_rate = np.exp(-r*dt)
    
    # Asset Price at Maturity:
    S_T = S0*d**(np.arange(N,-1,-1))*u**(np.arange(0,N+1,1))
    
    # Payout at Maturity:
    if call:
        payout = np.maximum(S_T-K,0)
    else:
        payout = np.maximum(K-S_T,0)
    
    # Go backwards through tree to discount
    disc_payout = payout
    for i in np.arange(N, 0, -1):
        disc_payout = disct_rate * (p_u*disc_payout[1:i+1] + p_d*disc_payout[0:i])
    
    return disc_payout[0]

In [3]:
binomial_european(S0=1, K=1, T=1, N=10, v=0.1, p_u=1/2, r=0)

0.12745146665293

In [4]:
binomial_european(S0=1, K=0.5, T=1, N=10, v=0.1, p_u=1/2, r=0)

0.5008688439266606

#### Q2. Finding Implied v

In [24]:
def binomial_european_minus_opval(S0, K, T, N, x, p_u, r, option_value):
    """
    This function returns binomial european option value minus provided option value
    """
    return binomial_european(S0, K, T, N, x, p_u, r)-option_value

def calibrate_v(option_value, S0, K, T, N, p_u=1/2, r=0, call=True):
    """
    Returns implied v based on provided option value
    """
    sol = root(lambda x: binomial_european_minus_opval(S0, K, T, N, x, p_u, r, option_value), x0=0.1, method='lm')
    return sol.x[0]

In [25]:
calibrate_v(0.12745146665293, S0=1, K=1, T=1, N=10, p_u=1/2, r=0)

0.1

In [26]:
calibrate_v(0.5008688439266606, S0=1, K=0.5, T=1, N=10, p_u=1/2, r=0)

0.1

#### Q3. American Call Option

In [27]:
def binomial_american(S0, K, T, N, v, p_u=1/2, r=0, call=True):
    """
    Returns American Option Values
    Parameters:
        S0: initial underlying price
        K: strike
        T: time until maturity
        N: number of time steps
        v: used to calculate up or down factor; (1+v) or (1-v) respectively 
        p_u: probability of up factor
        r: interest rate (in decimals)
        call: True for call option, False for put option
    """
    # check if 0 < v < 1:
    if v <= 0 or v >= 1:
        raise Exception('v should be >0 and <1')

    # check if 0 < p_u < 1:
    if p_u <= 0 or p_u >= 1:
        raise Exception('p_u should be >0 and <1')
    
    # compute constants used for pricing:
    dt = T/N
    u = 1+v
    d = 1-v
    p_d = 1 - p_u
    disct_rate = np.exp(-r*dt)
    
    # Asset Price at Maturity:
    S_T = S0*d**(np.arange(N,-1,-1))*u**(np.arange(0,N+1,1))
    
    # Payout at Maturity:
    if call:
        payout = np.maximum(S_T-K, 0)
    else:
        payout = np.maximum(K-S_T, 0)
    
    # Go backwards through tree to discount
    payout_t = payout
    for i in np.arange(N-1, -1, -1):
        S_t = S0*d**(np.arange(i,-1,-1))*u**(np.arange(0,i+1,1))
        payout_t[:i+1] = disct_rate * (p_u*payout_t[1:i+2] + p_d*payout_t[0:i+1])
        payout_t = payout_t[:-1]
        if call:
            payout_t = np.maximum(S_t-K, payout_t)
        else:
            payout_t = np.maximum(K-S_t, payout_t)
    return payout_t[0]

In [28]:
binomial_american(S0=1, K=1, T=1, N=10, v=0.1, p_u=1/2, r=0)

0.12745146665293

#### Q4. Expectation of max $S_{j}$

In [29]:
def get_max_Sj(S0, N, v):
    """
    returns expectation of maxium asset value
    parameters:
        S0: initial stock
        N: number of timesteps
        v: upfactor
    """
    return np.max(S0*(1-v)**(np.arange(N,-1,-1))*(1+v)**(np.arange(0,N+1,1)))

In [30]:
get_max_Sj(1,30,0.1)

17.44940226888645

#### Q5. Non-constant Vs

In [31]:
def calibrate_vjs(S0, K, T, N, N_option_prices=[], p_u=1/2, r=0, call=True):
    
    # check if len(N_option_prices) == N:
    if len(N_option_prices) != N:
        raise Exception('Option prices length should be N')
    
    # check if 0 < p_u < 1:
    if p_u <= 0 or p_u >= 1:
        raise Exception('p_u should be >0 and <1')
    
    # calibrate v (from t=0 to t=j)
    maturity_j = 0
    timestep_j = 0
    vj_list = []
    for price in N_option_prices:
        maturity_j += T/N
        timestep_j += 1
        vj_list.append(calibrate_v(price, S0, K, maturity_j, timestep_j, p_u, r, call))
        
    return vj_list

In [32]:
calibrate_vjs(S0=1, K=1, T=1, N=10, p_u=1/2, r=0, N_option_prices=[0.10,0.12,0.15,0.17,0.17,0.10,0.12,0.15,0.17,0.17], call=True)

[0.2000000000000001,
 0.21655250605964393,
 0.2027793946151302,
 0.20843453186051514,
 0.18554805663395177,
 0.10220742702022609,
 0.11107455997465751,
 0.1310053585718462,
 0.14190000884223083,
 0.13272413704978506]